In [1]:
import pickle

#google colabのとき↓
#/content/drive/MyDrive/dataset/
#研究室PCのとき↓
#/Users/Ma-Lab-PC3/SatoMaho/

def read_pickle(file_name):
    with open("/Users/admin/卒業研究/CNN/dataset/" + file_name + ".pickle", "rb") as f:
        dic = pickle.load(f)
    return dic["x"], dic["y"]

In [4]:
import numpy as np

In [2]:
import sys
print(sys.version)

3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]


In [ ]:
x, y = read_pickle("5sproecgshort10")
print(x.shape, y.shape)

for i in range(len(x) - 1, -1, -1):
    if np.isnan(x[i]).any() == True:
        x = np.delete(x, i, axis = 0)
        y = np.delete(y, i)

p = 0
for i in range(0, len(x)):
    f = 0
    for j in range(0, len(x[i])):
        if np.isnan(x[i][j]) == True:
            f += 1
    if f >= 1:
        p += 1
print(p)

In [ ]:
x0 = 0
x1 = 0
for x in range(0, len(y)):
    if y[x] == 0:
        x0 += 1
    else:
        x1 += 1
print(x0, x1)

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#from sklearn.model_selection import train_test_split
from torchvision import transforms

In [ ]:
import sys 
sys.path.append('c:/users/admin/appdata/roaming/python/python39/site-packages') 
import torch

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, l):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(1, l, d_model)
        for pos in range(l):
            for i in range(0, d_model, 2):
                pe[0, pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[0, pos, i + 1] = math.cos(pos / 10000 ** ((2 * (i + 1) / d_model)))
        
        self.pe = pe.unsqueeze(0)
        self.pe.requires_grad = False

        def forward(self, x):
            ret = math.sqrt(self.d_model) * x + self.pe
            return ret

In [ ]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(1, 64, 33)
        self.conv2 = nn.Conv1d(64, 128, 17)
        self.conv3 = nn.Conv1d(128, 256, 9)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)

        self.cls = nn.Parameter(torch.zeros(1, 1, 256))

        self.positionembedding = PositionalEncoding(256, 529)
        #self.positionEmbedding = nn.Parameter(torch.zeros(1, 529, 256))

        encoderLayer = nn.TransformerEncoderLayer(d_model = 256, nhead = 8, dropout = 0.2, batch_first = True)
        self.encoder = nn.TransformerEncoder(encoderLayer, num_layers = 2)

        self.linear = nn.Linear(256, 2)

    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(self.conv3(x))
        x = self.relu(x)
        x = torch.transpose(x, 2, 1)
        clsToken = self.cls.repeat_interleave(x.shape[0], dim = 0)
        x = torch.cat((clsToken, x), dim = 1)
        x = self.positionembedding(x)
        x = self.encoder(x)
        x = self.linear(x[:, 0, :])
        return x

In [9]:
class TransformerECG(nn.Module):
    def __init__(self):
        super(TransformerECG, self).__init__()
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(2)
        self.conv1 = nn.Conv1d(1, 64, 33) #65 33
        self.conv2 = nn.Conv1d(64, 128, 17) #33 17
        self.conv3 = nn.Conv1d(128, 256, 9) #17 9
        #self.dropout1 = nn.Dropout(0.3)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.cls = nn.Parameter(torch.zeros(1, 1, 256))
        self.positionembedding = PositionalEncoding(256, 297) #1241, 601, 561
        encoderLayer = nn.TransformerEncoderLayer(d_model = 256, nhead = 2, dropout = 0.1, batch_first = True)
        self.encoder = nn.TransformerEncoder(encoderLayer, num_layers = 2)
        self.maxpool2 = nn.MaxPool1d(2, ceil_mode = True)
        self.linear1 = nn.Linear(128, 512)
        self.linear2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = torch.transpose(x, 2, 1)
        clsToken = self.cls.expand(x.size(0), -1, -1) #clsToken = self.cls.repeat_interleave(x.shape[0], dim = 0)
        x = torch.cat((clsToken, x), dim = 1)
        x = self.positionembedding(x)
        x = self.encoder(x)
        x = self.maxpool2(x[:, 0, :])
        x = self.linear1(x)
        #x = self.dropout1(x)
        x = self.linear2(x)
        return x

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()

        self.dropout = nn.Dropout(0.4)

        self.conv1 = nn.Conv1d(1, 64, 17)
        self.conv2 = nn.Conv1d(64, 128, 17)
        self.conv3 = nn.Conv1d(128, 256, 17)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)

        self.fc1 = nn.Linear(16896, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.relu(x)
        x = self.pool(x)
        x = self.bn2(self.conv2(x))
        x = self.relu(x)
        x = self.pool(x)
        x = self.bn3(self.conv3(x))
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [10]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, label, transform = None):
        self.transform = transform
        self.data = data
        self.data_num = len(data)
        self.label = label

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        if self.transform:
            out_data = self.transform(self.data)[0][idx]
            out_label = self.label[idx]
        else:
            out_data = self.data[idx]
            out_label = self.label[idx]
        return out_data, out_label

In [12]:
import math

In [13]:
n = TransformerECG()

In [15]:
from torchsummary import summary

In [19]:
summary(n, (, 1, 1280))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: []

In [ ]:
x, y = read_pickle("5sproceedsubjectsP1")
print(x.shape)
transform = transforms.Compose([transforms.ToTensor()])
dataset = MyDataset(x, y, transform) #dataset[?][0]生体データ、dataset[?][1]ラベル
print(len(dataset[0]))
r = 0.7
train_size = int(len(dataset) * r)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(len(test_dataset))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 4, shuffle = True, drop_last = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 2, shuffle = True, drop_last = True)
print(len(test_dataloader.dataset))

## google colabのときのみ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
net = CNN()
net = net.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [ ]:
def train(dataloader, epoch, model, criterion, optimizer):
  size = len(dataloader.dataset)
  for e in range(epoch):
      train_loss_sum = 0
      train_acc_sum = 0
      val_loss_sum = 0
      val_acc_sum = 0
      model.train()
      for batch, (x, y) in enumerate(dataloader):
        x = x.unsqueeze(1)
        x, y = x.to(device, dtype = torch.float32), y.to(device, dtype = torch.long)

        optimizer.zero_grad()
        pred = model(x)
        #print(pred)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.item()
        pred = pred.argmax(1)
        train_acc_sum += pred.eq(y.view_as(pred)).sum().item()

      """model.eval()
      with torch.no_grad():
        for x, y in test_dataloader:
          x = x.unsqueeze(1)
          x, y = x.to(device, dtype = torch.float32), y.to(device, dtype = torch.long)
          pred = model(x)
          val_loss_sum += loss_fn(pred, y).item()
          pred = pred.argmax(1)
          val_acc_sum += pred.eq(y.view_as(pred)).sum().item()"""

      train_loss = train_loss_sum / len(dataloader)
      train_acc = train_acc_sum / train_size
      #val_loss = val_loss_sum / len(dataloader)
      #val_acc = val_acc_sum / test_size
      train_loss_list.append(train_loss)
      train_acc_list.append(train_acc)
      #val_loss_list.append(val_loss)
      #val_acc_list.append(val_acc)
      print(f"{e + 1} / {epoch} | Train Loss : {train_loss}, Acc : {100 * train_acc}")#, Val Loss : {val_loss}, Acc : {100 * val_acc}")

In [ ]:
def test(dataloader, model, loss_fn):
    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.unsqueeze(1)
            x, y = x.to(device, dtype = torch.float32), y.to(device, dtype = torch.long)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            pred = pred.argmax(1)
            correct += pred.eq(y.view_as(pred)).sum().item()
    print(f"Loss : {test_loss / len(dataloader)}, Accuracy : {100 * correct / test_size}%")

In [ ]:
epochs = 30
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []
train(train_dataloader, epochs, net, loss_fn, optimizer)
test(test_dataloader, net, loss_fn)
print("Done!")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_acc_list, label='Train Accuracy')
plt.plot(val_acc_list, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_loss_list, label='Train Loss')
plt.plot(val_loss_list, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [22]:
import sys
print(sys.version)

3.9.1 (tags/v3.9.1:1e5d33e, Dec  7 2020, 17:08:21) [MSC v.1927 64 bit (AMD64)]
